In [ ]:
import matplotlib
import os
import sys
sys.path.append('../')
from pyseus.plotting import pyseus_heatmap as phm
import pandas as pd
import plotly.offline
from plotly import graph_objs as go
import seaborn as sns

# Make notebook a full width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Import the imputed DF

In [ ]:
root = '../data/20210317_Organelle_IP_StartingMaterial_Bead_Amount/'
analysis = '20210520/'
filename = root + analysis + 'preprocessed_tables.pkl'

master = pd.read_pickle(filename)
imputed = master.bait_imputed_table

In [ ]:
# If you want to exclude some columns for plotting - this is the code

exclusion_df = pd.read_csv(root + analysis +  'plotting_exclusion_list.csv')

exclusion_list = exclusion_df[exclusion_df['plot']==False]['Baits'].to_list()
imputed.drop(exclusion_list, axis=1, inplace=True)

In [ ]:
# If you want to look at which samples are excluded from the plot
exclusion_list

### Transform prey intensities to (intensity - prey median)

In [ ]:
transformed = phm.subtract_prey_median(imputed ,mad_mod=True, mad_factor=1)

cols = list(set([x[0] for x in list(transformed)]))
cols.sort()
transformed = transformed[cols]

### If you DO NOT want the normalization, run the snippet below

In [ ]:
transformed = imputed.copy()

## Plotting Function

In [ ]:
# Adjust the min and max values as fit (None if you want default min/max)
zmin = 20
zmax = 36

#For median subtracted plot
fig, zmin, zmax, colormap = phm.color_map(transformed,zmin,zmax)

# Preview the colormap
fig.show()

### Clustering Preys and Baits

In [ ]:
# Cluster the preys and baits
prey_leaves = phm.prey_kmeans(transformed,k=50,method='average',ordering=True)
bait_leaves = phm.bait_leaves(transformed,method='average')

# Generate the heatmap (If you don't want baits clustered, assign bait_clust as False)

heatmap = phm.dendro_heatmap(transformed,prey_leaves,colormap,zmin,zmax,\
                             bait_leaves=bait_leaves,bait_clust=False)

### Plot the Figure!
comment and execute the lines when you're done  
otherwise loading the notebook again will be very slow


In [11]:
# Set up the Layout
# If you want prey / bait names, turn the ticks to True
layout = go.Layout({'width':1200, 'height':850},
                  xaxis={'showticklabels':True},
                 yaxis={'showticklabels':True})

# Plot!/
go.FigureWidget(heatmap,layout)